In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

import os
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
!unzip /content/drive/MyDrive/dataset.zip -d data

Archive:  /content/drive/MyDrive/dataset.zip
replace data/Hackathon_Dataset/train_images/CormorantGaramond-Regular_83.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
!ls data
!ls data/Hackathon_Dataset/train_images

Hackathon_Dataset
Alef-Bold_65.png			    DenkOne-Regular_65.png
Alef-Bold_66.png			    DenkOne-Regular_66.png
Alef-Bold_67.png			    DenkOne-Regular_67.png
Alef-Bold_68.png			    DenkOne-Regular_68.png
Alef-Bold_69.png			    DenkOne-Regular_69.png
Alef-Bold_70.png			    DenkOne-Regular_70.png
Alef-Bold_71.png			    DenkOne-Regular_71.png
Alef-Bold_72.png			    DenkOne-Regular_72.png
Alef-Bold_73.png			    DenkOne-Regular_73.png
Alef-Bold_74.png			    DenkOne-Regular_74.png
Alef-Bold_75.png			    DenkOne-Regular_75.png
Alef-Bold_76.png			    DenkOne-Regular_76.png
Alef-Bold_77.png			    DenkOne-Regular_77.png
Alef-Bold_78.png			    DenkOne-Regular_78.png
Alef-Bold_79.png			    DenkOne-Regular_79.png
Alef-Bold_80.png			    DenkOne-Regular_80.png
Alef-Bold_81.png			    DenkOne-Regular_81.png
Alef-Bold_82.png			    DenkOne-Regular_82.png
Alef-Bold_83.png			    DenkOne-Regular_83.png
Alef-Bold_84.png			    DenkOne-Regular_84.png
Alef-Bold_85.png			    DenkOne-Regular_85.png
Alef-Bold_86.png

In [6]:
dataset_dir = 'data/Hackathon_Dataset/train_images'


In [7]:
def load_images(directory, img_size=(28, 28)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=img_size, color_mode="grayscale")
            img_array = img_to_array(img)
            img_array = (img_array - 127.5) / 127.5
            images.append(img_array)
    return np.array(images)


In [8]:
images = load_images(dataset_dir)


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, LeakyReLU, Conv2DTranspose, Conv2D, Dropout, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input

In [10]:
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256 * 7 * 7, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 256)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(1, (7, 7), activation='tanh', padding='same'))
    return model

In [11]:
latent_dim = 100
generator = build_generator(latent_dim)
generator.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1266944   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 12544)             0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 14, 14, 128)       524416    
 anspose)                                                        
                                                                 
 batch_normalization (Batch  (None, 14, 14, 128)       512       
 Normalization)                                                  
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 128)       0

In [12]:
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

img_shape = (28, 28, 1)
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        640       
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [13]:
discriminator.trainable = False
z = Input(shape=(latent_dim,))
img = generator(z)
validity = discriminator(img)
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
combined.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         1926401   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 80769     
                                                                 
Total params: 2007170 (7.66 MB)
Trainable params: 1926017 (7.35 MB)
Non-trainable params: 81153 (317.00 KB)
_________________________________________________________________


In [14]:
def train(generator, discriminator, combined, epochs, batch_size, save_interval, latent_dim, images):
    X_train = images
    num_batches = X_train.shape[0]
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        for batch_idx in range(num_batches):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            gen_imgs = generator.predict(noise)
            d_loss_real = discriminator.train_on_batch(imgs, valid)
            d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = combined.train_on_batch(noise, valid)
        if epoch % save_interval == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
            save_imgs(generator, epoch, latent_dim)

In [15]:
def save_imgs(generator, epoch, latent_dim=100, examples=10, save_dir='generated_images'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    noise = np.random.normal(0, 1, (examples, latent_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(1, examples, figsize=(examples, 1))
    for i in range(examples):
        axs[i].imshow(gen_imgs[i, :, :, 0], cmap='gray')
        axs[i].axis('off')
    fig.suptitle(f'Generated Images (Epoch {epoch})')
    fig.savefig(f"{save_dir}/epoch_{epoch}.png")
    plt.close()

In [ ]:
epochs = 50
batch_size = 64
save_interval = 10

train(generator, discriminator, combined, epochs, batch_size, save_interval, latent_dim, images)

2/2 [==============================] - 0s 101ms/step
Epoch 0, D Loss: 0.10477238893508911, G Loss: 2.5014448165893555
2/2 [==============================] - 0s 103ms/step


In [ ]:
def generate_images(generator, latent_dim, n_images, save_dir='generated_alphabets'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    noise = np.random.normal(0, 1, (n_images, latent_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5
    for i in range(n_images):
        plt.imshow(gen_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
        plt.savefig(f"{save_dir}/alphabet_{i+1}.png")
        plt.close()

generate_images(generator, latent_dim, 26)